Importeren van modules

In [2]:
import pandas as pd
import pyodbc
import sqlite3
import numpy as np

Connectie maken met de  SSMS database

In [3]:
DB = {'servername': r'LAPTOP-NIPS1D14\SQLEXPRESS',
    'database': 'johari'}

ssms_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + 
                     ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
ssms_cursor = ssms_conn.cursor()
ssms_cursor

Microsoft Access database: go_staff

In [26]:
DB = {'servername': r'LAPTOP-NIPS1D14\SQLEXPRESS',
    'database': 'johari'}

ssms_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + 
                     ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
ssms_cursor = ssms_conn.cursor()
ssms_cursor

delete_query = """
DELETE FROM COUNTRY
DELETE FROM COURSE
DELETE FROM fPRODUCT
DELETE FROM fSALES_STAFF
DELETE FROM ORDER_METHOD
DELETE FROM ORDERS
DELETE FROM RETAILER_CONTACT
DELETE FROM RETURN_REASON
DELETE FROM RETURNED_ITEM
DELETE FROM rPRODUCT
DELETE FROM rSALES_STAFF
DELETE FROM rSATISFACTION_TYPE
DELETE FROM SALES_BRANCH
DELETE FROM SALES_INVENTORY_LEVELS
DELETE FROM SALES_PRODUCT_FORECAST
DELETE FROM SALES_TARGETData
DELETE FROM SATISFACTION
DELETE FROM SATISFACTION_TYPE
DELETE FROM TRAINING
"""
ssms_cursor.execute(delete_query)
ssms_conn.commit()

# Maak verbinding met de Access-database
go_staff_conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=../data/raw/go_staff.accdb;'
)
go_staff_conn = pyodbc.connect(go_staff_conn_str)

# Lijst met tabelnamen van de go_staff-database
go_staff_table_names = ['course', 'sales_branch', 'sales_staff', 'satisfaction', 'satisfaction_type', 'training']

# Dictionary om de dataframes op te slaan
go_staff = {}

# Lees de tabellen in en sla ze op in de dictionary
for table_name in go_staff_table_names:
    query = f'SELECT * FROM {table_name}'
    df = pd.read_sql(query, go_staff_conn)
    go_staff[table_name] = df

go_sales_conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=../data/raw/go_sales.accdb;'
)

go_sales_conn = pyodbc.connect(go_sales_conn_str)

# Lijst met tabelnamen van de go_sales-database
go_sales_table_names = ['country', 'order_details', 'order_header', 'order_method', 'product', 'product_line', 'product_type', 'retailer_site', 'return_reason', 'returned_item', 'sales_branch', 'sales_staff', 'SALES_TARGETData']

# Dictionary om de dataframes op te slaan
go_sales = {}

for table_name in go_sales_table_names:
    query = f'SELECT * FROM {table_name}'
    df = pd.read_sql(query, go_sales_conn)
    go_sales[table_name] = df

# Sla de gegevens binnen de dataframes op in een SQLite-database
sqlite_conn = sqlite3.connect('../data/raw/go_staff.db')

for table_name, df in go_staff.items():
    df.to_sql(table_name, sqlite_conn, index=False, if_exists='replace')


# Sla de gegevens binnen de dataframes op in de SQL Server-database
for index, row in go_staff['course'].iterrows():
    try:
        # Haal de namen van de kolommen op
        values = ', '.join([f"'{row[col]}'" if isinstance(row[col], str) else str(row[col]) for col in go_staff['course'].columns])

        # SQL-query
        query = f"INSERT INTO dbo.COURSE VALUES ({values})"
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        break

for index, row in go_staff['sales_branch'].iterrows():
    try:
        # Haal de namen van de kolommen op
        values = ', '.join([f"'{row[col]}'" if isinstance(row[col], str) else (str(row[col]) if row[col] is not None else 'NULL') for col in go_staff['sales_branch'].columns])        
        
        # SQL-query
        query = f"INSERT INTO dbo.SALES_BRANCH VALUES ({values})"
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        break

for index, row in go_staff['sales_staff'].iterrows():
    try:
        # Haal de namen van de kolommen op
        values = ', '.join([f"'{row[col]}'" if isinstance(row[col], str) else (str(row[col]) if row[col] is not None else 'NULL') for col in go_staff['sales_staff'].columns])        
        
        # Haal de bijbehorende sales_branch op
        sales_branch_code = row['SALES_BRANCH_CODE']
        sales_branch_row = go_staff['sales_branch'][go_staff['sales_branch']['SALES_BRANCH_CODE'] == sales_branch_code].iloc[0]

        # Haal de bijbehorende country op
        country_code = sales_branch_row['COUNTRY_CODE']
        country_row = go_sales['country'][go_sales['country']['COUNTRY_CODE'] == country_code].iloc[0]

        # Voeg de sales_branch en country gegevens toe aan de bestaande values
        values += f", '{sales_branch_row['ADDRESS1']}', '{sales_branch_row['ADDRESS2']}', '{sales_branch_row['CITY']}', '{sales_branch_row['REGION']}', '{sales_branch_row['POSTAL_ZONE']}', '{country_row['COUNTRY_CODE']}', '{country_row['COUNTRY']}', '{country_row['LANGUAGE']}', '{country_row['CURRENCY_NAME']}'"
        values = values.replace('None', 'NULL')
        
        # Voer de INSERT-query uit op de SSMS-database
        query = f"INSERT INTO dbo.rSALES_STAFF VALUES ({values})"
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        break

ssms_conn.close()

C:\Users\Jimmy\AppData\Local\Temp\ipykernel_12004\2331663370.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, go_staff_conn)
C:\Users\Jimmy\AppData\Local\Temp\ipykernel_12004\2331663370.py:67: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, go_sales_conn)


('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Incorrect syntax near 'Église'. (102) (SQLExecDirectW); [42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Unclosed quotation mark after the character string ')'. (105)")
'4', 'Denis', 'Pagé', 'Branch Manager', '+33 1 68 94 52 20', '325', '+33 1 68 94 56 60', 'DPage@grtd123.com', '03-Nov-1996 12:00:00 AM', '6', '18', '75, rue du Faubourg St-Honoré', 'NULL', 'Paris', 'NULL', 'F-75008', '1', 'France', 'EN', 'francs'
'5', 'Élizabeth', 'Michel', 'Level 3 Sales Representative', '+33 1 68 94 52 20', '336', '+33 1 68 94 56 60', 'EMichel@grtd123.com', '08-Jun-1995 12:00:00 AM', '6', '4', '75, rue du Faubourg St-Honoré', 'NULL', 'Paris', 'NULL', 'F-75008', '1', 'France', 'EN', 'francs'
'6', 'Émile', 'Clermont', 'Level 1 Sales Representative', '+33 1 68 94 52 20', '378', '+33 1 68 94 56 60', 'EClermont@grtd123.com', '07-Apr-1998 12:00:00 AM', '6', '4', '75, rue du Faubourg St-Honoré', 'NULL', 'Paris', 'NULL', 'F-75008', '1', '

Microsoft Access database: go_sales

In [48]:
go_sales_conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=../data/raw/go_sales.accdb;'
)

go_sales_conn = pyodbc.connect(go_sales_conn_str)

# Lijst met tabelnamen van de go_sales-database
go_sales_table_names = ['country', 'order_details', 'order_header', 'order_method', 'product', 'product_line', 'product_type', 'retailer_site', 'return_reason', 'returned_item', 'sales_branch', 'sales_staff', 'SALES_TARGETData']

# Dictionary om de dataframes op te slaan
go_sales = {}

for table_name in go_sales_table_names:
    query = f'SELECT * FROM {table_name}'
    df = pd.read_sql(query, go_sales_conn)
    go_sales[table_name] = df

go_sales['order_details'].head()

C:\Users\Jimmy\AppData\Local\Temp\ipykernel_16184\2663781144.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, go_sales_conn)


,ORDER_DETAIL_CODE,ORDER_NUMBER,PRODUCT_NUMBER,QUANTITY,UNIT_COST,UNIT_PRICE,UNIT_SALE_PRICE
0,48616,1258,17,92,60.29,84.41,78.37
1,48617,1352,17,190,60.29,84.41,67.08
2,48618,1544,17,82,60.29,84.41,78.37
3,48619,1249,17,190,60.29,84.41,72.26
4,48620,1268,17,152,60.29,84.41,78.05


Microsoft Access database: go_crm

In [100]:
go_crm_conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=../data/raw/go_crm.accdb;'
)

go_crm_conn = pyodbc.connect(go_crm_conn_str)

# Lijst met tabelnamen van de go_sales-database
go_crm_table_names = ['age_group', 'country', 'retailer', 'retailer_contact', 'retailer_headquarters', 'retailer_segment', 'retailer_site', 'retailer_type', 'sales_demographic', 'sales_territory']

# Dictionary om de dataframes op te slaan
go_crm = {}

for table_name in go_crm_table_names:
    query = f'SELECT * FROM {table_name}'
    df = pd.read_sql(query, go_crm_conn)
    go_crm[table_name] = df

go_crm['retailer'].head()

C:\Users\Jimmy\AppData\Local\Temp\ipykernel_16184\2555313751.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, go_crm_conn)


,RETAILER_CODE,RETAILER_CODEMR,COMPANY_NAME,RETAILER_TYPE_CODE
0,1,None,ActiForme,5
1,15,234,SportsClub,1
2,23,229,Anapurna,3
3,32,230,Cordages Discount,4
4,33,228,Altitudes extrêmes,6


Microsoft Excel bestand: GO_SALES_INVENTORY_LEVELSData

In [102]:
GO_SALES_INVENTORY_LEVELSData = pd.read_csv('../data/raw/GO_SALES_INVENTORY_LEVELSData.csv')
GO_SALES_INVENTORY_LEVELSData.head()

,INVENTORY_YEAR,INVENTORY_MONTH,PRODUCT_NUMBER,INVENTORY_COUNT
2021,4,48,1932,NaN
2021,4,49,1400,NaN
2021,4,50,21705,NaN
2021,4,51,9710,NaN
2021,4,52,5616,NaN


Microsft Excel bestand: GO_SALES_PRODUCT_FORECASTData

In [103]:
GO_SALES_PRODUCT_FORECASTData = pd.read_csv('../data/raw/GO_SALES_PRODUCT_FORECASTData.csv')
GO_SALES_PRODUCT_FORECASTData.head()

,PRODUCT_NUMBER,YEAR,MONTH,EXPECTED_VOLUME
0,44,2022,12,383
1,45,2021,1,80
2,45,2021,2,51
3,45,2021,3,214
4,45,2021,4,300


In [17]:
product = pd.read_excel('../data/raw/product.xlsx')


# Columns in the product dataframe: PRODUCT_NUMBER	INTRODUCTION_DATE	PRODUCT_TYPE_CODE	PRODUCTION_COST	MARGIN	PRODUCT_IMAGE	LANGUAGE	PRODUCT_NAME	DESCRIPTION
# Columns in the SSMS Database: product_product_id	product_name	product_image	product_line_name	product_type_name	product_introduction_date	product_cost	product_margin	product_language

for index, row in product.iterrows():
    try:
        introduction_date = pd.to_datetime(row['INTRODUCTION_DATE']).strftime('%Y-%m-%d')
        query = f"""
        INSERT INTO dbo.rPRODUCT 
        (product_product_id, product_name, product_image, product_line_name, product_type_name, product_introduction_date, product_cost, product_margin, product_language) 
        VALUES 
        ({row['PRODUCT_NUMBER']}, '{row['PRODUCT_NAME']}', '{row['PRODUCT_IMAGE']}', '', '{row['PRODUCT_TYPE_CODE']}', '{introduction_date}', '{row['PRODUCTION_COST']}', {row['MARGIN']}, '{row['LANGUAGE']}')
        """
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        break

# View the data in the SSMS Database
query = "SELECT * FROM dbo.rPRODUCT"
ssms_cursor.execute(query)
rows = ssms_cursor.fetchall()
df = pd.DataFrame.from_records(rows, columns=[desc[0] for desc in ssms_cursor.description])

C:\Users\Jimmy\AppData\Local\Temp\ipykernel_16184\14548683.py:8: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  introduction_date = pd.to_datetime(row['INTRODUCTION_DATE']).strftime('%Y-%m-%d')


Aanmaken van een SQLite database. Hier worden alle bewerkingen toegepast. Pas wanneer alles klaar is, worden alle gegevens binnen deze SQLite database overgezet naar de SSMS database

In [65]:
import datetime

# Voor het geval dat de database nog niet bestaat, maak een nieuwe SQLite-database aan
sqlite_conn = sqlite3.connect('../data/processed/johari_sqlite.db')

# Nieuwe dataframes met dummy data
dbo_product = pd.DataFrame({
    'PRODUCT_id': [1, 2, 3],
    'PRODUCT_name': ['Moondrop Variations', 'Sony IER-Z1R', 'Truthear Nova'],
    'PRODUCT_image': ['moondrop.jpg', 'sony.jpg', 'truthear.jpg'],
    'PRODUCT_LINE_name': ['Moondrop', 'Sony', 'Truthear'],
    'PRODUCT_TYPE_name': ['In-ear monitor', 'In-ear monitor', 'In-ear monitor'],
    'PRODUCT_INTRODUCTION_DATE_date': [datetime.date(2021, 1, 1), datetime.date(2021, 1, 1), datetime.date(2021, 1, 1)],
    'PRODUCT_COST_price': [400, 1100, 180],
    'PRODUCT_MARGIN_dimensions': [0.5, 0.6, 0.7],
    'PRODUCT_LANGUAGE_name': ['English', 'English', 'English'],
})

dbo_sales_staff = pd.DataFrame({
    'SALES_STAFF_id': [1, 2, 3],
    'SALES_STAFF_f_name': ['Karel', 'Henk', 'Regen'],
    'SALES_STAFF_l_name': ['Ankerwout', 'Tank', 'Achterstevoren'],
    'SALES_STAFF_phone': ['+316456789', '+316654321', '+316123123'],
    'SALES_STAFF_email': ['k.anker@yandex.ru', 'henk@yahoo.com', 'nee@ger.com'],
    'SALES_STAFF_fax': ['+316456789', '+316654321', '+316123123'],
    'SALES_STAFF_sales_branch_id': [1, 2, 3],
    'SALES_STAFF_POSITION_EN_name': ['Sales Manager', 'Sales Manager', 'Sales Manager'],
    'SALES_STAFF_DATE_HIRED_date': [datetime.date(2021, 1, 1), datetime.date(2021, 1, 1), datetime.date(2021, 1, 1)],
    'SALES_STAFF_EXTENSION_extension': ['+316456789', '+316654321', '+316123123'],
    'SALES_STAFF_SALES_BRANCH_address1': ['Kerkstraat 1', 'Kerkstraat 2', 'Kerkstraat 3'],
    'SALES_STAFF_SALES_BRANCH_address2': ['Kerkstraat 1', 'Kerkstraat 2', 'Kerkstraat 3'],
    'SALES_STAFF_SALES_BRANCH_region': ['Noord', 'Zuid', 'Oost'],
    'SALES_STAFF_SALES_BRANCH_city': ['Amsterdam', 'Rotterdam', 'Utrecht'],
    'SALES_STAFF_SALES_BRANCH_postal_zone': ['1000AA', '2000BB', '3000CC'],
    'SALES_STAFF_COUNTRY_id': [1, 2, 3],
    'SALES_STAFF_COUNTRY_name': ['Netherlands', 'Belgium', 'Germany'],
    'SALES_STAFF_COUNTRY_language': ['Dutch', 'French', 'German'],
    'SALES_STAFF_COUNTRY_currency': ['Euro', 'Euro', 'Euro'],
})

dbo_retailer_contact = pd.DataFrame({
    'retailer_contact_contact_id': np.random.randint(1, 100, size=10),
    'retailer_contact_site_id': np.random.randint(1, 100, size=10),
    'retailer_contact_first_name': np.random.choice(['John', 'Jane', 'Jim', 'Jill', 'Joe'], size=10),
    'retailer_contact_last_name': np.random.choice(['Smith', 'Johnson', 'Williams', 'Jones', 'Brown'], size=10),
    'retailer_contact_job_position_en': np.random.choice(['Manager', 'Employee', 'CEO', 'CFO', 'CTO'], size=10),
    'retailer_contact_extension': np.random.randint(1, 100, size=10),
    'retailer_contact_fax': np.random.randint(1, 100, size=10),
    'retailer_contact_email': [f'test{i}@test.com' for i in range(10)],
    'retailer_contact_gender': np.random.choice(['M', 'F'], size=10),
    'retailer_id': np.random.randint(1, 100, size=10),
    'retailer_codemr': np.random.randint(1, 100, size=10),
    'retailer_company_name': [f'Company{i}' for i in range(10)],
    'retailer_age_group_id': np.random.randint(1, 100, size=10),
    'retailer_age_group_upper_age': np.random.randint(30, 60, size=10),
    'retailer_age_group_lower_age': np.random.randint(20, 30, size=10),
    'retailer_sales_demographic_id': np.random.randint(1, 100, size=10),
    'retailer_sales_demographic_codemr': np.random.randint(1, 100, size=10),
    'retailer_sales_demographic_age_group_id': np.random.randint(1, 100, size=10),
    'retailer_sales_demographic_sales_percent': np.random.random(size=10),
    'retailer_sales_territory_id': np.random.randint(1, 100, size=10),
    'retailer_sales_territory_name_en': [f'Territory{i}' for i in range(10)],
    'retailer_country_id': np.random.randint(1, 100, size=10),
    'retailer_country_country_en': [f'Country{i}' for i in range(10)],
    'retailer_country_flag_image': [f'Image{i}' for i in range(10)],
    'retailer_country_sales_territory_id': np.random.randint(1, 100, size=10),
    'retailer_site_id': np.random.randint(1, 100, size=10),
    'retailer_site_retailer_id': np.random.randint(1, 100, size=10),
    'retailer_site_address1': [f'Address{i}' for i in range(10)],
    'retailer_site_address2': [f'Address{i}' for i in range(10)],
    'retailer_site_city': [f'City{i}' for i in range(10)],
    'retailer_site_region': [f'Region{i}' for i in range(10)],
    'retailer_site_postal_zone': [f'Postal{i}' for i in range(10)],
    'retailer_site_country_id': np.random.randint(1, 100, size=10),
    'retailer_site_active_indicator': np.random.choice([True, False], size=10),
    'retailer_headquarters_codemr': np.random.randint(1, 100, size=10),
    'retailer_headquarters_name': [f'Name{i}' for i in range(10)],
    'retailer_headquarters_address1': [f'Address{i}' for i in range(10)],
    'retailer_headquarters_address2': [f'Address{i}' for i in range(10)],
    'retailer_headquarters_city': [f'City{i}' for i in range(10)],
    'retailer_headquarters_region': [f'Region{i}' for i in range(10)],
    'retailer_headquarters_postal_zone': [f'Postal{i}' for i in range(10)],
    'retailer_headquarters_country_id': np.random.randint(1, 100, size=10),
    'retailer_headquarters_phone': [f'Phone{i}' for i in range(10)],
    'retailer_headquarters_fax': [f'Fax{i}' for i in range(10)],
    'retailer_headquarters_segment_id': np.random.randint(1, 100, size=10),
    'retailer_type_id': np.random.randint(1, 100, size=10),
    'retailer_type_type_en': [f'Type{i}' for i in range(10)],
    'retailer_segment_id': np.random.randint(1, 100, size=10),
    'retailer_segment_language': [f'Language{i}' for i in range(10)],
    'retailer_segment_name': [f'Segment Name{i}' for i in range(10)],
    'retailer_segment_description': [f'Segment Description{i}' for i in range(10)],
})

dbo_returned_reason = pd.DataFrame({
    'RETURNED_REASON_id': [1, 2, 3],
    'RETURNED_REASON_DESCRIPTION_name': ['Broken', 'Not satisfied', 'Wrong product'],
})

dbo_order_method = pd.DataFrame({
    'ORDER_METHOD_id': [1, 2, 3],
    'ORDER_METHOD_EN_name': ['Online', 'Phone', 'Mail'],
})

dbo_course = pd.DataFrame({
    'COURSE_id': [1, 2, 3],
    'COURSE_DESCRIPTION_en': ['Programming', 'Querying', 'Spreadsheets'],
})

dbo_satisfaction_type = pd.DataFrame({
    'SATISFACTION_TYPE_id': [1, 2, 3],
    'SATISFACTION_TYPE_DESCRIPTION_name': ['Good', 'Bad', 'Neutral'],
})


# Surrogate keys toevoegen aan de dataframes (plus beredeneringen)

# Wanneer deze dataframe samengevoegd wordt met externe bronnen, kan het zijn dat de PK niet meer uniek is of conflicts heeft.
dbo_product['PRODUCT_SK'] = list(range(1, len(dbo_product['PRODUCT_id']) + 1)) 

# Wanneer er herstructurering plaatsvindt in de organisatie, kan het zijn dat de PK niet meer uniek is of conflicts heeft.
dbo_sales_staff['SALES_STAFF_SK'] = list(range(1, len(dbo_sales_staff['SALES_STAFF_id']) + 1))

# Een contactpersoon kan van functie veranderen, maar het kan zijn dat je oude data wilt behouden voor bijvoorbeeld trendanalyses.
dbo_retailer_contact['retailer_contact_SK'] = list(range(1, len(dbo_retailer_contact['retailer_contact_contact_id']) + 1))

# Je kan in de loop van tijd de beredeneringen willen veranderen. Een voorbeeld hiervan is wanneer je een beschrijving wilt opsplitsen omdat het te vaag is, maar nog steeds de oorspronkelijke PK wilt gebruiken om aan te duiden dat de opgesplitste beschrijvingen bij elkaar horen. Dit mag natuurlijk niet, hierdoor wordt de SK nu gebruikt als de unieke kenmerk.
dbo_returned_reason['RETURNED_REASON_SK'] = list(range(1, len(dbo_returned_reason['RETURNED_REASON_id']) + 1))

# Hetzelfde als bij de vorige, maar in plaats van een beschrijving, kan het bijvoorbeeld zijn dat een order methode wordt opgesplitst (zoals Online = Webshop, App, etc.).
dbo_order_method['ORDER_METHOD_SK'] = list(range(1, len(dbo_order_method['ORDER_METHOD_id']) + 1))

# Hetzelfde als bij de vorige, maar in plaats van een beschrijving, kan het bijvoorbeeld zijn dat een course wordt opgesplitst (zoals Programming = Python, Java, etc.).
dbo_course['COURSE_SK'] = list(range(1, len(dbo_course['COURSE_id']) + 1))

# Hetzelfde als bij de vorige, maar in plaats van een beschrijving, kan het bijvoorbeeld zijn dat een satisfaction type wordt opgesplitst (zoals Good = Very good, Excellent, etc.).
dbo_satisfaction_type['SATISFACTION_TYPE_SK'] = list(range(1, len(dbo_satisfaction_type['SATISFACTION_TYPE_id']) + 1))


# Exporteer de dataframes naar de SQLite-database
dbo_product.to_sql('PRODUCT', sqlite_conn, if_exists='replace', index=False)
dbo_sales_staff.to_sql('SALES_STAFF', sqlite_conn, if_exists='replace', index=False)
dbo_retailer_contact.to_sql('RETAILER_CONTACT', sqlite_conn, if_exists='replace', index=False)
dbo_returned_reason.to_sql('RETURNED_REASON', sqlite_conn, if_exists='replace', index=False)
dbo_order_method.to_sql('ORDER_METHOD', sqlite_conn, if_exists='replace', index=False)
dbo_course.to_sql('COURSE', sqlite_conn, if_exists='replace', index=False)
dbo_satisfaction_type.to_sql('SATISFACTION_TYPE', sqlite_conn, if_exists='replace', index=False)

3

Zet alle gegevens om van de SQLite .db bestand naar de SSMS database

In [47]:
# Lees de data van een tabel uit de SQLite-database
product = pd.read_sql_query('SELECT * FROM PRODUCT', sqlite_conn)
sales_staff = pd.read_sql_query('SELECT * FROM SALES_STAFF', sqlite_conn)
retailer_contact = pd.read_sql_query('SELECT * FROM RETAILER_CONTACT', sqlite_conn)
returned_reason = pd.read_sql_query('SELECT * FROM RETURNED_REASON', sqlite_conn)
order_method = pd.read_sql_query('SELECT * FROM ORDER_METHOD', sqlite_conn)
course = pd.read_sql_query('SELECT * FROM COURSE', sqlite_conn)
satisfaction_type = pd.read_sql_query('SELECT * FROM SATISFACTION_TYPE', sqlite_conn)


# Itereer over de rijen van iedere dataframe en voeg elke record toe aan de SSMS-database

for index, row in product.iterrows():
    try:
        query = f"INSERT INTO dbo.rPRODUCT VALUES ({row['PRODUCT_id']}, '{row['PRODUCT_name']}', '{row['PRODUCT_image']}', '{row['PRODUCT_LINE_name']}', '{row['PRODUCT_TYPE_name']}', '{row['PRODUCT_INTRODUCTION_DATE_date']}', {row['PRODUCT_COST_price']}, {row['PRODUCT_MARGIN_dimensions']}, '{row['PRODUCT_LANGUAGE_name']}')"
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        break

for index, row in sales_staff.iterrows():
    try:
        query = f"INSERT INTO dbo.rSALES_STAFF VALUES ({row['SALES_STAFF_id']}, '{row['SALES_STAFF_f_name']}', '{row['SALES_STAFF_l_name']}', '{row['SALES_STAFF_phone']}', '{row['SALES_STAFF_email']}', '{row['SALES_STAFF_fax']}', {row['SALES_STAFF_sales_branch_id']}, '{row['SALES_STAFF_POSITION_EN_name']}', '{row['SALES_STAFF_DATE_HIRED_date']}', '{row['SALES_STAFF_EXTENSION_extension']}', '{row['SALES_STAFF_SALES_BRANCH_address1']}', '{row['SALES_STAFF_SALES_BRANCH_address2']}', '{row['SALES_STAFF_SALES_BRANCH_region']}', '{row['SALES_STAFF_SALES_BRANCH_city']}', '{row['SALES_STAFF_SALES_BRANCH_postal_zone']}', {row['SALES_STAFF_COUNTRY_id']}, '{row['SALES_STAFF_COUNTRY_name']}', '{row['SALES_STAFF_COUNTRY_language']}', '{row['SALES_STAFF_COUNTRY_currency']}')"
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        break

for index, row in retailer_contact.iterrows():
    try:
        query = f"INSERT INTO dbo.RETAILER_CONTACT VALUES ({row['retailer_contact_contact_id']}, {row['retailer_contact_site_id']}, '{row['retailer_contact_first_name']}', '{row['retailer_contact_last_name']}', '{row['retailer_contact_job_position_en']}', {row['retailer_contact_extension']}, {row['retailer_contact_fax']}, '{row['retailer_contact_email']}', '{row['retailer_contact_gender']}', {row['retailer_id']}, {row['retailer_codemr']}, '{row['retailer_company_name']}', {row['retailer_age_group_id']}, {row['retailer_age_group_upper_age']}, {row['retailer_age_group_lower_age']}, {row['retailer_sales_demographic_id']}, {row['retailer_sales_demographic_codemr']}, {row['retailer_sales_demographic_age_group_id']}, {row['retailer_sales_demographic_sales_percent']}, {row['retailer_sales_territory_id']}, '{row['retailer_sales_territory_name_en']}', {row['retailer_country_id']}, '{row['retailer_country_country_en']}', '{row['retailer_country_flag_image']}', {row['retailer_country_sales_territory_id']}, {row['retailer_site_id']}, {row['retailer_site_retailer_id']}, '{row['retailer_site_address1']}', '{row['retailer_site_address2']}', '{row['retailer_site_city']}', '{row['retailer_site_region']}', '{row['retailer_site_postal_zone']}', {row['retailer_site_country_id']}, {row['retailer_site_active_indicator']}, {row['retailer_headquarters_codemr']}, '{row['retailer_headquarters_name']}', '{row['retailer_headquarters_address1']}', '{row['retailer_headquarters_address2']}', '{row['retailer_headquarters_city']}', '{row['retailer_headquarters_region']}', '{row['retailer_headquarters_postal_zone']}', {row['retailer_headquarters_country_id']}, '{row['retailer_headquarters_phone']}', '{row['retailer_headquarters_fax']}', {row['retailer_headquarters_segment_id']}, {row['retailer_type_id']}, '{row['retailer_type_type_en']}', {row['retailer_segment_id']}, '{row['retailer_segment_language']}', '{row['retailer_segment_name']}', '{row['retailer_segment_description']}')"
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        break

for index, row in returned_reason.iterrows():
    try:
        query = f"INSERT INTO dbo.RETURN_REASON VALUES ({row['RETURNED_REASON_id']}, '{row['RETURNED_REASON_DESCRIPTION_name']}')"
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        break

for index, row in order_method.iterrows():
    try:
        query = f"INSERT INTO dbo.ORDER_METHOD VALUES ({row['ORDER_METHOD_id']}, '{row['ORDER_METHOD_EN_name']}')"
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        break

for index, row in course.iterrows():
    try:
        query = f"INSERT INTO dbo.COURSE VALUES ({row['COURSE_id']}, '{row['COURSE_DESCRIPTION_en']}')"
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        break

for index, row in satisfaction_type.iterrows():
    try:
        query = f"INSERT INTO dbo.SATISFACTION_TYPE VALUES ({row['SATISFACTION_TYPE_id']}, '{row['SATISFACTION_TYPE_DESCRIPTION_name']}')"
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        break

('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Violation of PRIMARY KEY constraint 'PK_rPRODUCT'. Cannot insert duplicate key in object 'dbo.rPRODUCT'. The duplicate key value is (1). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)")
'retailer_contact'
('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Violation of PRIMARY KEY constraint 'PK_RETURN_REASON'. Cannot insert duplicate key in object 'dbo.RETURN_REASON'. The duplicate key value is (1). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)")
('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Violation of PRIMARY KEY constraint 'PK_ORDER_METHOD'. Cannot insert duplicate key in object 'dbo.ORDER_METHOD'. The duplicate key value is (1). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been ter

In [70]:
pd.read_sql_query('SELECT * FROM dbo.rPRODUCT', ssms_conn)

C:\Users\Jimmy\AppData\Local\Temp\ipykernel_23344\395603908.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('SELECT * FROM dbo.rPRODUCT', ssms_conn)


,product_product_id,product_name,product_image,product_line_name,product_type_name,product_introduction_date,product_cost,product_margin,product_language
0,1,Moondrop Variations,moondrop.jpg,Moondrop,In-ear monitor,2021-01-01,400.0,0.5,English
1,2,Sony IER-Z1R,sony.jpg,Sony,In-ear monitor,2021-01-01,1100.0,0.6,English
2,3,Truthear Nova,truthear.jpg,Truthear,In-ear monitor,2021-01-01,180.0,0.7,English


Sluit connecties met alle databases

In [80]:
sqlite_conn.close()
ssms_conn.close()